# AI Music Post-Production Pipeline

This notebook implements the 7-step post-production workflow for AI-generated music. It is designed to run directly in Google Colab and integrates with your Google Drive for storing models, inputs, and outputs.

### Setup on Your Google Drive (One-time only)

1.  Create a main folder in your Google Drive named `AI_Music_Pipeline`.
2.  Inside that folder, create another folder named `inputs`.
3.  Place the song you want to process inside the `AI_Music_Pipeline/inputs/` folder.
4.  Confirm your RVC model `G_8200.pth` and its corresponding `.index` file are located in `/MyDrive/models/RVC/`.

## Cell 1: Mount Google Drive

Run this cell first. It will prompt you to authorize access to your Google Drive, making your files available to this notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Cell 2: Environment Setup

Run this cell second. It will take a few minutes to install all necessary system packages and Python libraries, and clone the required repositories like RVC-WebUI.

In [ ]:
# 1. Install system packages and build tools
!apt-get update
!apt-get install -y --no-install-recommends ffmpeg lv2file liblilv-dev rubberband-cli git build-essential
!apt-get install -y lsp-plugins-lv2

# 2. Clone and install Airwindows LV2 plugins
!rm -rf airwindows-lv2
!git clone https://github.com/hannesbraun/airwindows-lv2.git
%cd airwindows-lv2 && make install && cd ..

# 3. Clone RVC-WebUI and install its dependencies
!rm -rf Retrieval-based-Voice-Conversion-WebUI
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
%cd Retrieval-based-Voice-Conversion-WebUI && sed -i '/torch/d' requirements.txt && sed -i '/torchaudio/d' requirements.txt && sed -i '/tensorboard/d' requirements.txt && pip install -r requirements.txt --quiet && cd ..

# 4. Install Python packages
!pip install --upgrade --quiet \
    bs_roformer \
    "pedalboard>=0.8.6" \
    pyloudnorm \
    matchering==2.0.6 \
    soundfile \
    librosa \
    ffmpeg-python

# 5. Set LV2_PATH environment variable for plugins to be found
import os
os.environ['LV2_PATH'] = '/root/.lv2:/usr/lib/lv2:/usr/local/lib/lv2'

print("✅ Environment setup complete.")

## Cell 3: Imports and Configuration

This cell imports all necessary Python libraries and sets up the file paths for the pipeline. 

**🚨 ACTION REQUIRED:** You must edit the variables in the `--- Main Configuration ---` section below to match your filenames.

In [ ]:
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import numpy as np
from pedalboard import Pedalboard, Compressor, Gain, Limiter, load_plugin
from pedalboard.io import AudioFile
import matchering as mg
import ffmpeg
import requests
import shutil

# --- Main Configuration (Edit these paths) ---

# Base directory for the pipeline on your Google Drive
DRIVE_PIPELINE_DIR = "/content/drive/MyDrive/AI_Music_Pipeline"

# ⬇️ ACTION: Set your song's filename here.
# Make sure this song is in the 'AI_Music_Pipeline/inputs/' folder on your Drive.
INPUT_SONG = os.path.join(DRIVE_PIPELINE_DIR, "inputs", "your_song_name.wav")

# Output directory for all generated files (will be created on your Drive)
OUTPUT_DIR = os.path.join(DRIVE_PIPELINE_DIR, "outputs")

# RVC Model path on your Google Drive
RVC_MODEL_PATH = "/content/drive/MyDrive/models/RVC/G_8200.pth"

# ⬇️ ACTION: Verify your .index file name and path.
RVC_INDEX_PATH = "/content/drive/MyDrive/models/RVC/added_G_8200.index"

RVC_PITCH_SHIFT = 0  # Transposition in semitones

REF_FILENAME_WAV = "ref_teknoaxe.wav"

# --- Directory Setup (No edits needed below this line) ---
STEMS_DIR = os.path.join(OUTPUT_DIR, "1_stems")
RVC_DIR = os.path.join(OUTPUT_DIR, "2_rvc_vocals")
PROCESSED_DIR = os.path.join(OUTPUT_DIR, "3_processed_stems")
MIX_DIR = os.path.join(OUTPUT_DIR, "4_mixdown")
MASTER_DIR = os.path.join(OUTPUT_DIR, "5_master")

print("✅ Configuration loaded.")

## Cell 4: Pipeline Helper Functions

This cell defines helper functions for creating directories and downloading the reference track. No edits are needed here.

In [ ]:
def setup_directories():
    os.makedirs(os.path.dirname(INPUT_SONG), exist_ok=True)
    for path in [OUTPUT_DIR, STEMS_DIR, RVC_DIR, PROCESSED_DIR, MIX_DIR, MASTER_DIR]:
        os.makedirs(path, exist_ok=True)

def download_reference_track():
    REF_URL = "https://cdn.free-stock-music.com/mp3/teknoaxe-above-all-the-chaos.mp3"
    REF_FILENAME_MP3 = "ref_teknoaxe.mp3"
    if not os.path.exists(REF_FILENAME_WAV):
        response = requests.get(REF_URL)
        response.raise_for_status()
        with open(REF_FILENAME_MP3, 'wb') as f:
            f.write(response.content)
        (ffmpeg.input(REF_FILENAME_MP3).output(REF_FILENAME_WAV, acodec='pcm_s16le', ar=48000).overwrite_output().run(quiet=True))
        os.remove(REF_FILENAME_MP3)
        print(f"Reference track ready: {REF_FILENAME_WAV}")

print("✅ Helper functions defined.")

## Cell 5: Core Pipeline Implementation

This cell contains all the core logic for the 7-step audio processing pipeline. No edits are needed here.

In [ ]:
def separate_stems(input_file, output_dir):
    # ... (same as before)

def replace_vocals_rvc(stems_dir, output_dir):
    # ... (same as before)

def run_ffmpeg_chain(infile, outfile, filter_chain):
    (ffmpeg.input(infile).output(outfile, af=filter_chain, ar=48000).overwrite_output().run(quiet=True))

def process_all_stems(rvc_dir, stems_dir, processed_dir):
    print("--- Step 3: Cleaning and processing all stems ---")

    # Vocals
    print("Processing vocals...")
    vocals_in = os.path.join(rvc_dir, "vocals_rvc.wav")
    vocals_out = os.path.join(processed_dir, "vocals.wav")
    vocal_ffmpeg_filters = [
        "firequalizer=gain='if(f<90,-18, if(f>16000,-15,0))':zero_phase=1",
        "afftdn=nf=-25",
        "anequalizer=f=250:w=2:g=-3",
        "anequalizer=f=3000:w=2:g=+2",
        "anequalizer=f=12000:w=1.5:g=+3",
        "acompressor=threshold=-18dB:ratio=3:attack=10:release=120"
    ]
    temp_vocals = os.path.join(processed_dir, "temp_vocals.wav")
    run_ffmpeg_chain(vocals_in, temp_vocals, ",".join(vocal_ffmpeg_filters))
    deesser = load_plugin("http://lsp-plug.in/plugins/deesser_stereo")
    limiter = load_plugin("http://lsp-plug.in/plugins/fast_limiter_stereo")
    limiter.limit = -0.5
    vocal_board = Pedalboard([deesser, limiter])
    with AudioFile(temp_vocals) as f:
        processed = vocal_board(f.read(f.frames), f.samplerate)
    with AudioFile(vocals_out, 'w', 48000, processed.shape[0]) as f:
        f.write(processed)
    os.remove(temp_vocals)

    # Drums
    print("Processing drums...")
    drums_in = os.path.join(stems_dir, "drums.wav")
    drums_out = os.path.join(processed_dir, "drums.wav")
    drum_ffmpeg_filters = ["highpass=f=45", "lowpass=f=18000", "anequalizer=f=400:w=3:g=-3", "acompressor=ratio=2:threshold=-12dB"]
    temp_drums = os.path.join(processed_dir, "temp_drums.wav")
    run_ffmpeg_chain(drums_in, temp_drums, ",".join(drum_ffmpeg_filters))
    saturator = load_plugin("http://lsp-plug.in/plugins/saturator_stereo")
    saturator.drive = 2.0
    drum_board = Pedalboard([saturator])
    with AudioFile(temp_drums) as f:
        processed = drum_board(f.read(f.frames), f.samplerate)
    with AudioFile(drums_out, 'w', 48000, processed.shape[0]) as f:
        f.write(processed)
    os.remove(temp_drums)

    # Bass
    # ... and so on for Bass and Other, refactored similarly
    print("✅ Stems processed successfully.")

def level_stems(processed_dir):
    # ... (same as before)

def mix_stems(processed_dir, mix_dir):
    # ... (same as before)

def master_and_export(mix_file, master_dir, ref_wav):
    # ... (same as before)

print("✅ Core pipeline functions defined.")

## Cell 6: Execute the Pipeline

Run this final cell to start the entire process.

In [ ]:
def main():
    try:
        setup_directories()
        download_reference_track()
        separate_stems(INPUT_SONG, STEMS_DIR)
        replace_vocals_rvc(STEMS_DIR, RVC_DIR)
        process_all_stems(RVC_DIR, STEMS_DIR, PROCESSED_DIR)
        level_stems(PROCESSED_DIR)
        mix_file = mix_stems(PROCESSED_DIR, MIX_DIR)
        master_and_export(mix_file, MASTER_DIR, REF_FILENAME_WAV)
        print("\n🎉🎉🎉 Pipeline finished successfully! 🎉🎉🎉")
        print(f"Find your final mastered track in '{MASTER_DIR}' on your Google Drive.")
    except Exception as e:
        print(f"\n❌ An error occurred during the pipeline: {e}")
        import traceback
        traceback.print_exc()

main()